In [1]:
from datetime import datetime
from pydantic import BaseModel

# Definimos una clase que representa una entrada de log del sistema
class LogEntry(BaseModel):
    timestamp: datetime       # Fecha y hora de la medición
    sala: str                 # Sala donde se registró la medición (ej: Sala_1)
    estado: str               # Nivel de severidad del log (ej: "INFO", "WARNING")
    temperatura: float        # Temperatura medida (en grados Celsius)
    humedad: float            # Humedad relativa (%)
    co2: int                  # Concentración de CO2 medida (en ppm)
    mensaje: str              # Descripción del evento (texto libre)

    # Método especial que permite comparar dos objetos LogEntry usando el operador <
    def __lt__(self, other: 'LogEntry'):
        """Permite comparar objetos LogSensor por su timestamp."""
        assert isinstance(other, LogEntry), NotImplemented
        return self.timestamp < other.timestamp


    @staticmethod
    def from_csv_row(row: dict) -> 'LogEntry':
        """Crea una instancia de LogSensor a partir de una fila de CSV."""
        return LogEntry(
            timestamp=datetime.fromisoformat(row['timestamp']),
            sala=row['sala'],
            estado=row['estado'],
            temperatura=float(row['temperatura']),
            humedad=float(row['humedad']),
            co2=int(row['co2']),
            mensaje=row['mensaje']
        )

    # Clase interna de configuración de Pydantic
    class Config:
        frozen = True  # Hace que los objetos sean inmutables


In [4]:
from typing import List
from pydantic import BaseModel
from model.log_entry import LogEntry

class LogList(BaseModel):
    logs: List[LogEntry]

In [8]:
from typing import Any, Dict, List, Optional
from datetime import datetime
from pydantic import BaseModel, ValidationError, root_validator

# Definimos los campos clave esperados
CAMPOS_CLAVE = {'timestamp', 'sala', 'estado', 'temperatura', 'humedad', 'co2', 'mensaje'}

class LogEntry(BaseModel):
    timestamp: datetime
    sala: str
    estado: str
    temperatura: float
    humedad: float
    co2: int
    mensaje: str

    @root_validator(pre=True)
    def validar_campos_clave(cls, values):
        missing = CAMPOS_CLAVE - set(values.keys())
        if missing:
            raise ValueError(f'Faltan campos clave: {missing}')
        return values

    @staticmethod
    def from_source_row(row: dict) -> Optional['LogEntry']:
        try:
            return LogEntry(
                timestamp=datetime.fromisoformat(row['timestamp']),
                sala=row['sala'],
                estado=row['estado'],
                temperatura=float(row['temperatura']),
                humedad=float(row['humedad']),
                co2=int(row['co2']),
                mensaje=row['mensaje']
            )
        except (KeyError, ValueError, TypeError, ValidationError) as e:
            print(f"Registro mal formado descartado: {e}")
            return None

class LogList(BaseModel):
    logs: List[LogEntry]

# Simulación de fuente de logs en memoria (podría ser CSV, JSON, etc.)
def cargar_logs_desde_fuente(fuente: List[Dict[str, Any]]) -> LogList:
    logs_validos = []
    for fila in fuente:
        log = LogEntry.from_source_row(fila)
        if log:
            logs_validos.append(log)
    return LogList(logs=logs_validos)

# Ejemplo de uso con datos en memoria (simulando una fuente CSV)
fuente_ejemplo = [
    {
        'timestamp': '2024-06-10T14:30:00',
        'sala': 'Sala_1',
        'estado': 'INFO',
        'temperatura': '23.5',
        'humedad': '45.2',
        'co2': '800',
        'mensaje': 'Medición normal'
    },
    {
        'timestamp': '2024-06-10T15:00:00',
        'sala': 'Sala_2',
        'estado': 'WARNING',
        'temperatura': '25.1',
        'humedad': '40.0',
        'co2': '950',
        'mensaje': 'CO2 elevado'
    },
    # Registro mal formado (falta 'co2')
    {
        'timestamp': '2024-06-10T15:30:00',
        'sala': 'Sala_1',
        'estado': 'INFO',
        'temperatura': '22.8',
        'humedad': '47.0',
        'mensaje': 'Medición normal'
    }
]

log_list = cargar_logs_desde_fuente(fuente_ejemplo)
print(log_list)

Registro mal formado descartado: 'co2'
logs=[LogEntry(timestamp=datetime.datetime(2024, 6, 10, 14, 30), sala='Sala_1', estado='INFO', temperatura=23.5, humedad=45.2, co2=800, mensaje='Medición normal'), LogEntry(timestamp=datetime.datetime(2024, 6, 10, 15, 0), sala='Sala_2', estado='WARNING', temperatura=25.1, humedad=40.0, co2=950, mensaje='CO2 elevado')]


C:\Users\lalessi\AppData\Local\Temp\ipykernel_8708\3016647584.py:17: PydanticDeprecatedSince20: Pydantic V1 style `@root_validator` validators are deprecated. You should migrate to Pydantic V2 style `@model_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  @root_validator(pre=True)


In [6]:
# Creamos una fila de ejemplo como si viniera de un CSV
fila_ejemplo = {
    'timestamp': '2024-06-10T14:30:00',
    'sala': 'Sala_1',
    'estado': 'INFO',
    'temperatura': '23.5',
    'humedad': '45.2',
    'co2': '800',
    'mensaje': 'Medición normal'
}

# Creamos una instancia de LogEntry usando el método from_csv_row
log = LogEntry.from_csv_row(fila_ejemplo)

# Mostramos el objeto creado
print(log)

# Probamos la creación de una lista de logs usando LogList
logs = [log]  # Usamos la instancia 'log' ya creada
log_list = LogList(logs=logs)

# Mostramos el objeto LogList creado
print(log_list)

timestamp=datetime.datetime(2024, 6, 10, 14, 30) sala='Sala_1' estado='INFO' temperatura=23.5 humedad=45.2 co2=800 mensaje='Medición normal'
logs=[LogEntry(timestamp=datetime.datetime(2024, 6, 10, 14, 30), sala='Sala_1', estado='INFO', temperatura=23.5, humedad=45.2, co2=800, mensaje='Medición normal')]


In [ ]:
# Creamos varias filas de ejemplo como si vinieran de un CSV
filas_ejemplo = [
    {
        'timestamp': '2024-06-10T14:30:00',
        'sala': 'Sala_1',
        'estado': 'INFO',
        'temperatura': '23.5',
        'humedad': '45.2',
        'co2': '800',
        'mensaje': 'Medición normal'
    },
    {
        'timestamp': '2024-06-10T15:00:00',
        'sala': 'Sala_2',
        'estado': 'WARNING',
        'temperatura': '25.1',
        'humedad': '40.0',
        'co2': '950',
        'mensaje': 'CO2 elevado'
    },
    {
        'timestamp': '2024-06-10T15:30:00',
        'sala': 'Sala_1',
        'estado': 'INFO',
        'temperatura': '22.8',
        'humedad': '47.0',
        'co2': '780',
        'mensaje': 'Medición normal'
    }
]

# Creamos instancias de LogEntry usando el método from_csv_row
logs = [LogEntry.from_csv_row(fila) for fila in filas_ejemplo]

# Creamos un LogList con todas las instancias
log_list = LogList(logs=logs)

# Mostramos el objeto LogList creado
print(log_list)